## Finding good features

In Machine Learning, it is important to find good features that help to predict the target variables. Finding good features will help us to build better classifiers. But once we have trained a good model, we might also want to know which features were the most useful in classifing the instances. 




### Example: stability selection -  Training a classifier on the IRIS dataset

In this section, we will again train a classifier on the [IRIS data set](http://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html) and use stability selection to find the strongest features.

<img src="pics/iris.png" width=300>
This data set about plants is included in sklearn and already ready to use, meaning that features are already extracted for the data instances x, and each training instance has an associated class label y. 
The iris data set consists of 150 training instances with 3 classes (setosa,versicolor,virginica). Lets train a classifier and evaluate it.

In [2]:
from sklearn import datasets
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression, RandomizedLogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# 3 classes, 150 instances: X=iris[’data’]  y=iris[’target’]
iris = datasets.load_iris()
indices = np.random.permutation(len(iris['data']))
# split in 80% train, 20% test
len_test = int(len(iris['data'])*0.2)
# train part (all except test part)
X_train = iris['data'][indices[:-len_test]]
y_train = iris['target'][indices[:-len_test]]
# test part
X_test = iris['data'][indices[-len_test:]]
y_test = iris['target'][indices[-len_test:]]
# output statistics
print("#inst train: %s" % (len(X_train)))
print("#inst test: %s" % (len(X_test)))
# learn knn classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
print(clf)
y_pred= clf.predict(X_test)
print("Pred:", y_pred)
print("Gold:", y_test)
# get accuracies
print(classification_report(y_test, y_pred, target_names=iris['target_names']))
print(confusion_matrix(y_test, y_pred))

#inst train: 120
#inst test: 30
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Pred: [0 2 1 2 2 2 2 1 0 0 0 1 1 0 0 0 2 1 2 2 0 0 1 1 0 0 0 1 0 2]
Gold: [0 2 1 2 2 2 2 1 0 0 0 1 1 0 0 0 2 1 2 2 0 0 1 1 0 0 0 1 0 2]
             precision    recall  f1-score   support

     setosa       1.00      1.00      1.00        13
 versicolor       1.00      1.00      1.00         8
  virginica       1.00      1.00      1.00         9

avg / total       1.00      1.00      1.00        30

[[13  0  0]
 [ 0  8  0]
 [ 0  0  9]]


The classifier is doing a pretty good job. But which features were the most predictive ones? To answer this question, we will look at stability selection. 

### Stability selection
Stability selection [(Meinshausen & Bühlmann, 2009)](http://stat.ethz.ch/~nicolai/stability.pdf) is a relatively novel model for feature selection. It is based on subsampling in combination with a learning algorithm. The main idea is to apply a feature selection algorithm on different subset of data using different subsets of features. This procedure is then iterated. At the end, strong features will appear in many subsamples, since they will often be selected. The output of stability selection is a probability score per feature, which indicates how likely it is that this feature is selected.

Stability selection is implemented in `sklearn` in `RandomizedLogisticRegression`. It trains several LogisticRegression models on subsets of the data, using a feature selection algorithm (L1 regularisation). The algorithm records how often the feature has been selected, and hence the final score gives an indication of importance of a feature for a given task.

From the `sklearn` documentation:
"Randomized Logistic Regression works by subsampling the training data and fitting a L1-penalized LogisticRegression model where the penalty of a random subset of coefficients has been scaled. By performing this double randomization several times, the method assigns high scores to features that are repeatedly selected across randomizations. This is known as stability selection. In short, features selected more often are considered good features."


In [46]:
# lets use stabiliy selection
randomLogReg = RandomizedLogisticRegression()
randomLogReg.fit(X_train, y_train)

names = iris['feature_names']
print("Features sorted by their score:")
for score, name in sorted(zip(map(lambda x: round(x, 4), randomLogReg.scores_), names), reverse=True):
    print(score, name)



Features sorted by their score:
0.685 sepal width (cm)
0.64 petal width (cm)
0.635 petal length (cm)
0.145 sepal length (cm)


We now see that petal length is amongst the most predictive features. However, we do not see how predictive it was for each class. For this, we inspect the coefficient per class, as shown next.

In [47]:
n=2
feature_names = iris['feature_names']
for class_num in range(0,len(clf.coef_)):
    coefs_with_fns = sorted(zip(clf.coef_[class_num], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    print("class_num",class_num)
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

class_num 0
	-2.1721	petal length (cm)		1.3932	sepal width (cm)
	-1.0098	petal width (cm)		0.4121	sepal length (cm)
class_num 1
	-1.5053	sepal width (cm)		0.5530	petal length (cm)
	-1.0418	petal width (cm)		0.3650	sepal length (cm)
class_num 2
	-1.5483	sepal length (cm)		2.2953	petal width (cm)
	-1.3097	sepal width (cm)		2.1763	petal length (cm)


### Exercise 1:

The output 'class_num 0' probably doesn't tell you much. Add the name of the current target label to the function above (hint: 'target_names'). Now inspect the most predictive features and look at the various types of iris, do the most predictive features intuitively make sense?

### Exercise 2:

Extend the sentiment classification example, find the most predictive feature per class.

Hint: a method you can use is given below.

In [48]:
def show_most_informative_features(vectorizer, clf, n=10):
    feature_names = vectorizer.get_feature_names()
    for i in range(0,len(clf.coef_)):
        coefs_with_fns = sorted(zip(clf.coef_[i], feature_names))
        top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
        print("i",i)
        for (coef_1, fn_1), (coef_2, fn_2) in top:
            print("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

## References

There is much more to be said about feature selection than what we cover here. Below are some pointers.

* http://machinelearningmastery.com/an-introduction-to-feature-selection/
* http://scikit-learn.org/stable/datasets/twenty_newsgroups.html
* http://blog.datadive.net/selecting-good-features-part-iv-stability-selection-rfe-and-everything-side-by-side/

In [49]:
# note: in sklearn 0.17 I got a deprecated warning for RandomizedLogisticRegression, upgrading solved it.
import sklearn
sklearn.__version__

'0.18'

In [4]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], 
      dtype='<U10')